In [1]:
# ----------------------------------------------------------------------------
# Author        :    Vasileios Perifanis
# Affiliation   :    Euclid team, Democritus University of Thrace, Dept. of Electrical & Computer Engineering
# Created Date  :    03/2022
# version       :    1.0
# ---------------------------------------------------------------------------

In [2]:
import numpy as np # linear algebra
import pandas as pd # data analysis
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn import preprocessing


import torch
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv("datasets/titanic_train.csv", header=0) # read csv

data['Initial']=0
for i in data:
    data['Initial']=data.Name.str.extract('([A-Za-z]+)\.') #extract the Salutations

# Assigning the NaN Values the mean ages
data.loc[(data.Age.isnull())&(data.Initial=='Mr'),'Age']=33
data.loc[(data.Age.isnull())&(data.Initial=='Mrs'),'Age']=36
data.loc[(data.Age.isnull())&(data.Initial=='Master'),'Age']=5
data.loc[(data.Age.isnull())&(data.Initial=='Miss'),'Age']=22
data.loc[(data.Age.isnull())&(data.Initial=='Other'),'Age']=46

data['Embarked'].fillna('S',inplace=True)

data['Age_band']=0
data.loc[data['Age']<=16,'Age_band']=0
data.loc[(data['Age']>16)&(data['Age']<=32),'Age_band']=1
data.loc[(data['Age']>32)&(data['Age']<=48),'Age_band']=2
data.loc[(data['Age']>48)&(data['Age']<=64),'Age_band']=3
data.loc[data['Age']>64,'Age_band']=4

data['Family_Size']=0
data['Family_Size']=data['Parch']+data['SibSp']#family size
data['Alone']=0
data.loc[data.Family_Size==0,'Alone']=1#Alone

data['Fare_Range']=pd.qcut(data['Fare'],5)
data['Fare_cat']=0
data.loc[data['Fare']<=7.91,'Fare_cat']=0
data.loc[(data['Fare']>7.91)&(data['Fare']<=14.454),'Fare_cat']=1
data.loc[(data['Fare']>14.454)&(data['Fare']<=31),'Fare_cat']=2
data.loc[(data['Fare']>31)&(data['Fare']<=513),'Fare_cat']=3

data['Sex'].replace(['male','female'],[0,1],inplace=True)
data['Embarked'].replace(['S','C','Q'],[0,1,2],inplace=True)
data['Initial'].replace(['Mr','Mrs','Miss','Master','Other'],[0,1,2,3,4],inplace=True)

data.drop(['Name','Age','Ticket','Fare','Cabin','Initial','Fare_Range','PassengerId'],axis=1,inplace=True)

data.head()

,Survived,Pclass,Sex,SibSp,Parch,Embarked,Age_band,Family_Size,Alone,Fare_cat
0,0,3,0,1,0,0,1,1,0,0
1,1,1,1,1,0,1,2,1,0,3
2,1,3,1,0,0,0,1,0,1,1
3,1,1,1,1,0,0,2,1,0,3
4,0,3,0,0,0,0,2,0,1,1


In [4]:
def train_sk(train_X, train_Y, test_X, test_Y):
    # initialize classifier
    classifier = MLPClassifier(hidden_layer_sizes=(32,32,32), random_state=0, max_iter=500)

    # train the classifier
    classifier.fit(train_X,train_Y)

    # make predictions on train/test set
    y_pred_train = classifier.predict(train_X)
    y_pred = classifier.predict(test_X)

    # calculate scores
    accuracy_train = accuracy_score(y_pred_train, train_Y)
    precision_train = precision_score(y_pred_train, train_Y)
    recall_train = recall_score(y_pred_train, train_Y)
    f1_train = f1_score(y_pred_train, train_Y)

    accuracy = accuracy_score(y_pred, test_Y)
    precision = precision_score(y_pred, test_Y)
    recall = recall_score(y_pred, test_Y)
    f1 = f1_score(y_pred, test_Y)

    print(f"[Training set] Accuracy: {accuracy_train:.4f}, Precision: {precision_train:.4f}, Recall: {recall_train:.4f}, F1: {f1_train:.4f}\n",
          f"[Testing set] Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    return accuracy, precision, recall, f1

In [5]:
def get_Xy_to_numpy(dataset):
    X = dataset[dataset.columns[1:]].to_numpy()
    y = dataset[dataset.columns[:1]].to_numpy().ravel()
    return X, y

# K Fold cross-validation

In [6]:
from sklearn.model_selection import KFold

In [7]:
kf = KFold(n_splits=10, shuffle=True, random_state=0)

In [8]:
X, y = get_Xy_to_numpy(data)
min_max_scaler = preprocessing.MinMaxScaler().fit(X)
X = min_max_scaler.transform(X)

In [9]:
accuracy, precision, recall, f1 = dict(), dict(), dict(), dict()
for i, (train_idx, test_idx) in enumerate(kf.split(X)):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    if i == 0:
        print(len(X_train), len(X_test))
    
    acc, prec, rec, f = train_sk(X_train, y_train, X_test, y_test)
    accuracy[i] = acc
    precision[i] = prec
    recall[i] = rec
    f1[i] = f

801 90
[Training set] Accuracy: 0.8165, Precision: 0.7261, Recall: 0.7746, F1: 0.7496
 [Testing set] Accuracy: 0.8111, Precision: 0.7436, Recall: 0.8056, F1: 0.7733
[Training set] Accuracy: 0.8105, Precision: 0.7019, Recall: 0.7878, F1: 0.7424
 [Testing set] Accuracy: 0.7865, Precision: 0.6667, Recall: 0.6897, F1: 0.6780
[Training set] Accuracy: 0.8192, Precision: 0.6977, Recall: 0.8097, F1: 0.7496
 [Testing set] Accuracy: 0.7865, Precision: 0.6452, Recall: 0.7143, F1: 0.6780
[Training set] Accuracy: 0.8292, Precision: 0.7614, Recall: 0.7845, F1: 0.7728
 [Testing set] Accuracy: 0.7865, Precision: 0.6667, Recall: 0.7742, F1: 0.7164
[Training set] Accuracy: 0.8117, Precision: 0.6286, Recall: 0.8534, F1: 0.7239
 [Testing set] Accuracy: 0.8652, Precision: 0.7037, Recall: 0.8261, F1: 0.7600
[Training set] Accuracy: 0.8130, Precision: 0.6720, Recall: 0.8132, F1: 0.7359
 [Testing set] Accuracy: 0.7865, Precision: 0.6774, Recall: 0.7000, F1: 0.6885
[Training set] Accuracy: 0.8030, Precision: 0

In [10]:
import statistics

In [11]:
def get_statistics(accuracy, precision, recall, f):
    print(f"Stdev:\n\tAcc: {statistics.stdev(accuracy):.4f}\n\tPre: {statistics.stdev(precision):.4f}\n\tRec: {statistics.stdev(recall):.4f}\n\tF1: {statistics.stdev(f):.4f}")
    print(f"Mean Scores:\n\tAcc: {statistics.mean(accuracy):.4f}\n\tPre: {statistics.mean(precision):.4f}\n\tRec: {statistics.mean(recall):.4f}\n\tF1: {statistics.mean(f):.4f}")
    return statistics.mean(f)

In [12]:
mean_f_score = get_statistics(list(accuracy.values()), list(precision.values()), list(recall.values()), list(f1.values()))

Stdev:
	Acc: 0.0261
	Pre: 0.0562
	Rec: 0.0820
	F1: 0.0334
Mean Scores:
	Acc: 0.7980
	Pre: 0.6600
	Rec: 0.7856
	F1: 0.7131


# Stratified K Fold cross-validation

In [13]:
from sklearn.model_selection import StratifiedKFold

In [14]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

In [15]:
accuracy, precision, recall, f1 = dict(), dict(), dict(), dict()
for i, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    if i == 0:
        print(len(X_train), len(X_test))
    
    acc, prec, rec, f = train_sk(X_train, y_train, X_test, y_test)
    accuracy[i] = acc
    precision[i] = prec
    recall[i] = rec
    f1[i] = f

801 90
[Training set] Accuracy: 0.8015, Precision: 0.6906, Recall: 0.7681, F1: 0.7273
 [Testing set] Accuracy: 0.8667, Precision: 0.8286, Recall: 0.8286, F1: 0.8286
[Training set] Accuracy: 0.8080, Precision: 0.6461, Recall: 0.8156, F1: 0.7210
 [Testing set] Accuracy: 0.7978, Precision: 0.6176, Recall: 0.8077, F1: 0.7000
[Training set] Accuracy: 0.8130, Precision: 0.6688, Recall: 0.8110, F1: 0.7331
 [Testing set] Accuracy: 0.7865, Precision: 0.6471, Recall: 0.7586, F1: 0.6984
[Training set] Accuracy: 0.8130, Precision: 0.6948, Recall: 0.7926, F1: 0.7405
 [Testing set] Accuracy: 0.8876, Precision: 0.8235, Recall: 0.8750, F1: 0.8485
[Training set] Accuracy: 0.8317, Precision: 0.6916, Recall: 0.8419, F1: 0.7594
 [Testing set] Accuracy: 0.8202, Precision: 0.7353, Recall: 0.7812, F1: 0.7576
[Training set] Accuracy: 0.8092, Precision: 0.7370, Recall: 0.7592, F1: 0.7479
 [Testing set] Accuracy: 0.7865, Precision: 0.7647, Recall: 0.7027, F1: 0.7324
[Training set] Accuracy: 0.8180, Precision: 0

In [16]:
mean_f_score_skf = get_statistics(list(accuracy.values()), list(precision.values()), list(recall.values()), list(f1.values()))

Stdev:
	Acc: 0.0412
	Pre: 0.0965
	Rec: 0.0529
	F1: 0.0665
Mean Scores:
	Acc: 0.8069
	Pre: 0.6840
	Rec: 0.7860
	F1: 0.7286


In [17]:
class ClassifierDataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return len(self.x_data)
    
def generate_dataset(train_x, train_y, batch_size, shuffle=True):
    dataset = ClassifierDataset(torch.from_numpy(train_x).float(), torch.from_numpy(train_y).long())
    loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=shuffle)

    return loader

class MLP(torch.nn.Module):
    def __init__(self, input_dim, num_classes):
        super(MLP, self).__init__()
        self.layer1 = torch.nn.Linear(input_dim, 256)
        self.layer2 = torch.nn.Linear(256, 128)
        self.out = torch.nn.Linear(128, num_classes)
        self.activation = torch.nn.ReLU()

    def forward(self, x):
        x = self.layer1(x)
        x = self.activation(x)
        x = self.layer2(x)
        x = self.activation(x)
        x = self.out(x)
        return x

def test(model, data, criterion):
    model.eval()
    y_true, y_pred = [], []
    loss, correct = 0., 0
    with torch.no_grad():
        for x, y in data:
            out = model(x)
            loss += criterion(out, y).item()
            pred = out.argmax(dim=1)

            # correct += int((pred == y).sum().item()) # torch accuracy method
            for i in range(len(pred)):
                y_pred.append(pred[i].item())
            for i in range(len(y)):
                y_true.append(y[i].item())
            
    loss = loss / len(data)
    #accuracy = correct / len(data.dataset) # torch accuracy method
    accuracy = accuracy_score(y_pred, y_true)
    precision = precision_score(y_pred, y_true)
    recall = recall_score(y_pred, y_true)
    f1 = f1_score(y_pred, y_true)

    return loss, accuracy, precision, recall, f1

def train(model, train_loader, test_loader, optimizer, criterion, epochs, log_interval=1):
    for epoch in range(1, epochs + 1):
        model.train()  # training mode
        for data, target in train_loader:
            optimizer.zero_grad()  # zero out gradients
            y_pred = model(data)  # forward propagation
            loss = criterion(y_pred, target)  # calculate loss
            loss.backward()  # backward pass, compute grads wrt to model parameters
            optimizer.step()  # update weights

        train_loss, train_acc, train_prec, train_rec, train_f = test(model, train_loader, criterion)  # test the model
        test_loss, test_acc, test_pred, test_rec, test_f = test(model, test_loader, criterion)

        if epoch % log_interval == 0:  # log
            print(f"Epoch: [{epoch}/{epochs}]: Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}, Precision: {train_prec:.4f}, Recall: {train_rec:.4f}, F1: {train_f:.4f}\n")
            print(f"\tLoss: {test_loss:.4f}, Eval Accuracy: {test_acc:.4f}, Eval Precision: {train_prec:.4f}, Eval Recall: {train_rec:.4f}, Eval F1: {train_f:.4f}")


    return test_acc, test_pred, test_rec, test_f

In [18]:
epochs = 30

accuracy, precision, recall, f1 = dict(), dict(), dict(), dict()

for i, (train_idx, test_idx) in enumerate(kf.split(X)):
    
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    num_classes = len(np.unique(y_train))
    model = MLP(X_train.shape[1], num_classes)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()
    
    if i == 0:
        print(len(X_train), len(X_test))
        
    train_loader = generate_dataset(X_train, y_train, batch_size=1024, shuffle=True)
    test_loader = generate_dataset(X_test, y_test, batch_size=1, shuffle=False)
    
    acc, prec, rec, f = train(model, train_loader, test_loader, optimizer, criterion, epochs, log_interval=1)
    accuracy[i] = acc
    precision[i] = prec
    recall[i] = rec
    f1[i] = f

801 90
Epoch: [1/30]: Loss: 0.6585, Accuracy: 0.6217, Precision: 0.0000, Recall: 0.0000, F1: 0.0000

	Loss: 0.6685, Eval Accuracy: 0.5667, Eval Precision: 0.0000, Eval Recall: 0.0000, Eval F1: 0.0000
Epoch: [2/30]: Loss: 0.6390, Accuracy: 0.6217, Precision: 0.0000, Recall: 0.0000, F1: 0.0000

	Loss: 0.6510, Eval Accuracy: 0.5667, Eval Precision: 0.0000, Eval Recall: 0.0000, Eval F1: 0.0000
Epoch: [3/30]: Loss: 0.6220, Accuracy: 0.6217, Precision: 0.0000, Recall: 0.0000, F1: 0.0000

	Loss: 0.6352, Eval Accuracy: 0.5667, Eval Precision: 0.0000, Eval Recall: 0.0000, Eval F1: 0.0000
Epoch: [4/30]: Loss: 0.6069, Accuracy: 0.6654, Precision: 0.1221, Recall: 0.9487, F1: 0.2164

	Loss: 0.6206, Eval Accuracy: 0.6222, Eval Precision: 0.1221, Eval Recall: 0.9487, Eval F1: 0.2164
Epoch: [5/30]: Loss: 0.5928, Accuracy: 0.6841, Precision: 0.1716, Recall: 0.9630, F1: 0.2913

	Loss: 0.6058, Eval Accuracy: 0.6333, Eval Precision: 0.1716, Eval Recall: 0.9630, Eval F1: 0.2913
Epoch: [6/30]: Loss: 0.5793,

Epoch: [20/30]: Loss: 0.4639, Accuracy: 0.7918, Precision: 0.6763, Recall: 0.7617, F1: 0.7165

	Loss: 0.4477, Eval Accuracy: 0.7753, Eval Precision: 0.6763, Eval Recall: 0.7617, Eval F1: 0.7165
Epoch: [21/30]: Loss: 0.4603, Accuracy: 0.7918, Precision: 0.6763, Recall: 0.7617, F1: 0.7165

	Loss: 0.4461, Eval Accuracy: 0.7753, Eval Precision: 0.6763, Eval Recall: 0.7617, Eval F1: 0.7165
Epoch: [22/30]: Loss: 0.4572, Accuracy: 0.7918, Precision: 0.6763, Recall: 0.7617, F1: 0.7165

	Loss: 0.4451, Eval Accuracy: 0.7753, Eval Precision: 0.6763, Eval Recall: 0.7617, Eval F1: 0.7165
Epoch: [23/30]: Loss: 0.4545, Accuracy: 0.7918, Precision: 0.6763, Recall: 0.7617, F1: 0.7165

	Loss: 0.4449, Eval Accuracy: 0.7753, Eval Precision: 0.6763, Eval Recall: 0.7617, Eval F1: 0.7165
Epoch: [24/30]: Loss: 0.4520, Accuracy: 0.7930, Precision: 0.6763, Recall: 0.7645, F1: 0.7177

	Loss: 0.4453, Eval Accuracy: 0.7753, Eval Precision: 0.6763, Eval Recall: 0.7645, Eval F1: 0.7177
Epoch: [25/30]: Loss: 0.4497, 

Epoch: [3/30]: Loss: 0.6261, Accuracy: 0.6185, Precision: 0.0000, Recall: 0.0000, F1: 0.0000

	Loss: 0.6478, Eval Accuracy: 0.5955, Eval Precision: 0.0000, Eval Recall: 0.0000, Eval F1: 0.0000
Epoch: [4/30]: Loss: 0.6131, Accuracy: 0.6185, Precision: 0.0000, Recall: 0.0000, F1: 0.0000

	Loss: 0.6401, Eval Accuracy: 0.5955, Eval Precision: 0.0000, Eval Recall: 0.0000, Eval F1: 0.0000
Epoch: [5/30]: Loss: 0.6013, Accuracy: 0.6746, Precision: 0.1536, Recall: 0.9592, F1: 0.2648

	Loss: 0.6334, Eval Accuracy: 0.6404, Eval Precision: 0.1536, Eval Recall: 0.9592, Eval F1: 0.2648
Epoch: [6/30]: Loss: 0.5899, Accuracy: 0.6833, Precision: 0.1765, Recall: 0.9643, F1: 0.2983

	Loss: 0.6269, Eval Accuracy: 0.6404, Eval Precision: 0.1765, Eval Recall: 0.9643, Eval F1: 0.2983
Epoch: [7/30]: Loss: 0.5785, Accuracy: 0.6833, Precision: 0.1765, Recall: 0.9643, F1: 0.2983

	Loss: 0.6202, Eval Accuracy: 0.6404, Eval Precision: 0.1765, Eval Recall: 0.9643, Eval F1: 0.2983
Epoch: [8/30]: Loss: 0.5668, Accura

Epoch: [20/30]: Loss: 0.4604, Accuracy: 0.7918, Precision: 0.7175, Recall: 0.7434, F1: 0.7302

	Loss: 0.4399, Eval Accuracy: 0.7753, Eval Precision: 0.7175, Eval Recall: 0.7434, Eval F1: 0.7302
Epoch: [21/30]: Loss: 0.4575, Accuracy: 0.7943, Precision: 0.7079, Recall: 0.7534, F1: 0.7300

	Loss: 0.4336, Eval Accuracy: 0.7753, Eval Precision: 0.7079, Eval Recall: 0.7534, Eval F1: 0.7300
Epoch: [22/30]: Loss: 0.4550, Accuracy: 0.7943, Precision: 0.7079, Recall: 0.7534, F1: 0.7300

	Loss: 0.4286, Eval Accuracy: 0.7753, Eval Precision: 0.7079, Eval Recall: 0.7534, Eval F1: 0.7300
Epoch: [23/30]: Loss: 0.4527, Accuracy: 0.7905, Precision: 0.6921, Recall: 0.7543, F1: 0.7219

	Loss: 0.4251, Eval Accuracy: 0.7753, Eval Precision: 0.6921, Eval Recall: 0.7543, Eval F1: 0.7219
Epoch: [24/30]: Loss: 0.4507, Accuracy: 0.7918, Precision: 0.6921, Recall: 0.7569, F1: 0.7231

	Loss: 0.4228, Eval Accuracy: 0.7753, Eval Precision: 0.6921, Eval Recall: 0.7569, Eval F1: 0.7231
Epoch: [25/30]: Loss: 0.4489, 

Epoch: [7/30]: Loss: 0.5641, Accuracy: 0.7544, Precision: 0.3725, Recall: 0.9580, F1: 0.5365

	Loss: 0.5773, Eval Accuracy: 0.7865, Eval Precision: 0.3725, Eval Recall: 0.9580, Eval F1: 0.5365
Epoch: [8/30]: Loss: 0.5506, Accuracy: 0.7594, Precision: 0.3856, Recall: 0.9593, F1: 0.5501

	Loss: 0.5654, Eval Accuracy: 0.7865, Eval Precision: 0.3856, Eval Recall: 0.9593, Eval F1: 0.5501
Epoch: [9/30]: Loss: 0.5375, Accuracy: 0.7594, Precision: 0.4248, Recall: 0.8844, F1: 0.5740

	Loss: 0.5539, Eval Accuracy: 0.7865, Eval Precision: 0.4248, Eval Recall: 0.8844, Eval F1: 0.5740
Epoch: [10/30]: Loss: 0.5249, Accuracy: 0.7643, Precision: 0.4902, Recall: 0.8197, F1: 0.6135

	Loss: 0.5430, Eval Accuracy: 0.7865, Eval Precision: 0.4902, Eval Recall: 0.8197, Eval F1: 0.6135
Epoch: [11/30]: Loss: 0.5128, Accuracy: 0.7793, Precision: 0.5621, Recall: 0.8000, F1: 0.6603

	Loss: 0.5331, Eval Accuracy: 0.7865, Eval Precision: 0.5621, Eval Recall: 0.8000, Eval F1: 0.6603
Epoch: [12/30]: Loss: 0.5016, Acc

Epoch: [28/30]: Loss: 0.4409, Accuracy: 0.7955, Precision: 0.6913, Recall: 0.7597, F1: 0.7239

	Loss: 0.4264, Eval Accuracy: 0.8315, Eval Precision: 0.6913, Eval Recall: 0.7597, Eval F1: 0.7239
Epoch: [29/30]: Loss: 0.4397, Accuracy: 0.7955, Precision: 0.6945, Recall: 0.7579, F1: 0.7248

	Loss: 0.4242, Eval Accuracy: 0.8315, Eval Precision: 0.6945, Eval Recall: 0.7579, Eval F1: 0.7248
Epoch: [30/30]: Loss: 0.4384, Accuracy: 0.8005, Precision: 0.6945, Recall: 0.7687, F1: 0.7297

	Loss: 0.4223, Eval Accuracy: 0.8315, Eval Precision: 0.6945, Eval Recall: 0.7687, Eval F1: 0.7297
Epoch: [1/30]: Loss: 0.6748, Accuracy: 0.6222, Precision: 0.0000, Recall: 0.0000, F1: 0.0000

	Loss: 0.6877, Eval Accuracy: 0.5618, Eval Precision: 0.0000, Eval Recall: 0.0000, Eval F1: 0.0000
Epoch: [2/30]: Loss: 0.6571, Accuracy: 0.6222, Precision: 0.0000, Recall: 0.0000, F1: 0.0000

	Loss: 0.6745, Eval Accuracy: 0.5618, Eval Precision: 0.0000, Eval Recall: 0.0000, Eval F1: 0.0000
Epoch: [3/30]: Loss: 0.6421, Acc

Epoch: [14/30]: Loss: 0.4963, Accuracy: 0.7930, Precision: 0.5467, Recall: 0.8454, F1: 0.6640

	Loss: 0.5224, Eval Accuracy: 0.7416, Eval Precision: 0.5467, Eval Recall: 0.8454, Eval F1: 0.6640
Epoch: [15/30]: Loss: 0.4876, Accuracy: 0.7805, Precision: 0.5733, Recall: 0.7818, F1: 0.6615

	Loss: 0.5087, Eval Accuracy: 0.7191, Eval Precision: 0.5733, Eval Recall: 0.7818, Eval F1: 0.6615
Epoch: [16/30]: Loss: 0.4800, Accuracy: 0.7781, Precision: 0.5733, Recall: 0.7748, F1: 0.6590

	Loss: 0.4963, Eval Accuracy: 0.7416, Eval Precision: 0.5733, Eval Recall: 0.7748, Eval F1: 0.6590
Epoch: [17/30]: Loss: 0.4732, Accuracy: 0.7706, Precision: 0.5867, Recall: 0.7458, F1: 0.6567

	Loss: 0.4855, Eval Accuracy: 0.7640, Eval Precision: 0.5867, Eval Recall: 0.7458, Eval F1: 0.6567
Epoch: [18/30]: Loss: 0.4674, Accuracy: 0.7768, Precision: 0.6233, Recall: 0.7391, F1: 0.6763

	Loss: 0.4763, Eval Accuracy: 0.7528, Eval Precision: 0.6233, Eval Recall: 0.7391, Eval F1: 0.6763
Epoch: [19/30]: Loss: 0.4623, 

In [19]:
mean_f_score_torch = get_statistics(list(accuracy.values()), list(precision.values()), list(recall.values()), list(f1.values()))

Stdev:
	Acc: 0.0236
	Pre: 0.0528
	Rec: 0.0900
	F1: 0.0462
Mean Scores:
	Acc: 0.7991
	Pre: 0.6820
	Rec: 0.7695
	F1: 0.7193


In [20]:
epochs = 30

accuracy, precision, recall, f1 = dict(), dict(), dict(), dict()

for i, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    num_classes = len(np.unique(y_train))
    model = MLP(X_train.shape[1], num_classes)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()
    
    if i == 0:
        print(len(X_train), len(X_test))
        
    train_loader = generate_dataset(X_train, y_train, batch_size=1024, shuffle=True)
    test_loader = generate_dataset(X_test, y_test, batch_size=1, shuffle=False)
    
    acc, prec, rec, f = train(model, train_loader, test_loader, optimizer, criterion, epochs, log_interval=1)
    accuracy[i] = acc
    precision[i] = prec
    recall[i] = rec
    f1[i] = f

801 90
Epoch: [1/30]: Loss: 0.6699, Accuracy: 0.7578, Precision: 0.4495, Recall: 0.8466, F1: 0.5872

	Loss: 0.6695, Eval Accuracy: 0.7667, Eval Precision: 0.4495, Eval Recall: 0.8466, Eval F1: 0.5872
Epoch: [2/30]: Loss: 0.6510, Accuracy: 0.7291, Precision: 0.3094, Recall: 0.9500, F1: 0.4668

	Loss: 0.6496, Eval Accuracy: 0.7444, Eval Precision: 0.3094, Eval Recall: 0.9500, Eval F1: 0.4668
Epoch: [3/30]: Loss: 0.6350, Accuracy: 0.7266, Precision: 0.2964, Recall: 0.9681, F1: 0.4539

	Loss: 0.6323, Eval Accuracy: 0.7333, Eval Precision: 0.2964, Eval Recall: 0.9681, Eval F1: 0.4539
Epoch: [4/30]: Loss: 0.6205, Accuracy: 0.7341, Precision: 0.3192, Recall: 0.9608, F1: 0.4792

	Loss: 0.6164, Eval Accuracy: 0.7444, Eval Precision: 0.3192, Eval Recall: 0.9608, Eval F1: 0.4792
Epoch: [5/30]: Loss: 0.6061, Accuracy: 0.7403, Precision: 0.3388, Recall: 0.9541, F1: 0.5000

	Loss: 0.6004, Eval Accuracy: 0.7444, Eval Precision: 0.3388, Eval Recall: 0.9541, Eval F1: 0.5000
Epoch: [6/30]: Loss: 0.5916,

Epoch: [15/30]: Loss: 0.4910, Accuracy: 0.7681, Precision: 0.5779, Recall: 0.7607, F1: 0.6568

	Loss: 0.4584, Eval Accuracy: 0.7865, Eval Precision: 0.5779, Eval Recall: 0.7607, Eval F1: 0.6568
Epoch: [16/30]: Loss: 0.4839, Accuracy: 0.7830, Precision: 0.6331, Recall: 0.7617, F1: 0.6915

	Loss: 0.4507, Eval Accuracy: 0.7978, Eval Precision: 0.6331, Eval Recall: 0.7617, Eval F1: 0.6915
Epoch: [17/30]: Loss: 0.4775, Accuracy: 0.7830, Precision: 0.6331, Recall: 0.7617, F1: 0.6915

	Loss: 0.4439, Eval Accuracy: 0.7978, Eval Precision: 0.6331, Eval Recall: 0.7617, Eval F1: 0.6915
Epoch: [18/30]: Loss: 0.4720, Accuracy: 0.7843, Precision: 0.6364, Recall: 0.7626, F1: 0.6938

	Loss: 0.4381, Eval Accuracy: 0.7978, Eval Precision: 0.6364, Eval Recall: 0.7626, Eval F1: 0.6938
Epoch: [19/30]: Loss: 0.4671, Accuracy: 0.7843, Precision: 0.6591, Recall: 0.7491, F1: 0.7012

	Loss: 0.4332, Eval Accuracy: 0.7978, Eval Precision: 0.6591, Eval Recall: 0.7491, Eval F1: 0.7012
Epoch: [20/30]: Loss: 0.4630, 

Epoch: [28/30]: Loss: 0.4407, Accuracy: 0.7968, Precision: 0.7078, Recall: 0.7491, F1: 0.7279

	Loss: 0.4130, Eval Accuracy: 0.8539, Eval Precision: 0.7078, Eval Recall: 0.7491, Eval F1: 0.7279
Epoch: [29/30]: Loss: 0.4393, Accuracy: 0.8005, Precision: 0.7078, Recall: 0.7569, F1: 0.7315

	Loss: 0.4133, Eval Accuracy: 0.8539, Eval Precision: 0.7078, Eval Recall: 0.7569, Eval F1: 0.7315
Epoch: [30/30]: Loss: 0.4379, Accuracy: 0.8005, Precision: 0.7078, Recall: 0.7569, F1: 0.7315

	Loss: 0.4136, Eval Accuracy: 0.8539, Eval Precision: 0.7078, Eval Recall: 0.7569, Eval F1: 0.7315
Epoch: [1/30]: Loss: 0.6606, Accuracy: 0.6197, Precision: 0.0097, Recall: 1.0000, F1: 0.0193

	Loss: 0.6583, Eval Accuracy: 0.6067, Eval Precision: 0.0097, Eval Recall: 1.0000, Eval F1: 0.0193
Epoch: [2/30]: Loss: 0.6404, Accuracy: 0.6534, Precision: 0.1006, Recall: 0.9688, F1: 0.1824

	Loss: 0.6347, Eval Accuracy: 0.6292, Eval Precision: 0.1006, Eval Recall: 0.9688, Eval F1: 0.1824
Epoch: [3/30]: Loss: 0.6226, Acc

Epoch: [11/30]: Loss: 0.5130, Accuracy: 0.7781, Precision: 0.5130, Recall: 0.8495, F1: 0.6397

	Loss: 0.5293, Eval Accuracy: 0.8090, Eval Precision: 0.5130, Eval Recall: 0.8495, Eval F1: 0.6397
Epoch: [12/30]: Loss: 0.5012, Accuracy: 0.7731, Precision: 0.5552, Recall: 0.7917, F1: 0.6527

	Loss: 0.5202, Eval Accuracy: 0.7978, Eval Precision: 0.5552, Eval Recall: 0.7917, Eval F1: 0.6527
Epoch: [13/30]: Loss: 0.4906, Accuracy: 0.7743, Precision: 0.5812, Recall: 0.7749, F1: 0.6642

	Loss: 0.5122, Eval Accuracy: 0.7978, Eval Precision: 0.5812, Eval Recall: 0.7749, Eval F1: 0.6642
Epoch: [14/30]: Loss: 0.4812, Accuracy: 0.7756, Precision: 0.5942, Recall: 0.7689, F1: 0.6703

	Loss: 0.5056, Eval Accuracy: 0.7865, Eval Precision: 0.5942, Eval Recall: 0.7689, Eval F1: 0.6703
Epoch: [15/30]: Loss: 0.4730, Accuracy: 0.7868, Precision: 0.6558, Recall: 0.7566, F1: 0.7026

	Loss: 0.5004, Eval Accuracy: 0.7978, Eval Precision: 0.6558, Eval Recall: 0.7566, Eval F1: 0.7026
Epoch: [16/30]: Loss: 0.4661, 

Epoch: [25/30]: Loss: 0.4445, Accuracy: 0.7930, Precision: 0.6883, Recall: 0.7518, F1: 0.7186

	Loss: 0.4528, Eval Accuracy: 0.7640, Eval Precision: 0.6883, Eval Recall: 0.7518, Eval F1: 0.7186
Epoch: [26/30]: Loss: 0.4427, Accuracy: 0.7980, Precision: 0.6851, Recall: 0.7645, F1: 0.7226

	Loss: 0.4535, Eval Accuracy: 0.7753, Eval Precision: 0.6851, Eval Recall: 0.7645, Eval F1: 0.7226
Epoch: [27/30]: Loss: 0.4412, Accuracy: 0.8030, Precision: 0.6851, Recall: 0.7757, F1: 0.7276

	Loss: 0.4546, Eval Accuracy: 0.7865, Eval Precision: 0.6851, Eval Recall: 0.7757, Eval F1: 0.7276
Epoch: [28/30]: Loss: 0.4399, Accuracy: 0.8017, Precision: 0.6818, Recall: 0.7749, F1: 0.7254

	Loss: 0.4559, Eval Accuracy: 0.7753, Eval Precision: 0.6818, Eval Recall: 0.7749, Eval F1: 0.7254
Epoch: [29/30]: Loss: 0.4387, Accuracy: 0.8017, Precision: 0.6818, Recall: 0.7749, F1: 0.7254

	Loss: 0.4573, Eval Accuracy: 0.7753, Eval Precision: 0.6818, Eval Recall: 0.7749, Eval F1: 0.7254
Epoch: [30/30]: Loss: 0.4376, 

Epoch: [15/30]: Loss: 0.4992, Accuracy: 0.7855, Precision: 0.6331, Recall: 0.7677, F1: 0.6940

	Loss: 0.4688, Eval Accuracy: 0.7753, Eval Precision: 0.6331, Eval Recall: 0.7677, Eval F1: 0.6940
Epoch: [16/30]: Loss: 0.4915, Accuracy: 0.7868, Precision: 0.6396, Recall: 0.7665, F1: 0.6973

	Loss: 0.4601, Eval Accuracy: 0.7753, Eval Precision: 0.6396, Eval Recall: 0.7665, Eval F1: 0.6973
Epoch: [17/30]: Loss: 0.4844, Accuracy: 0.7905, Precision: 0.6558, Recall: 0.7652, F1: 0.7063

	Loss: 0.4518, Eval Accuracy: 0.7753, Eval Precision: 0.6558, Eval Recall: 0.7652, Eval F1: 0.7063
Epoch: [18/30]: Loss: 0.4779, Accuracy: 0.7905, Precision: 0.6721, Recall: 0.7555, F1: 0.7113

	Loss: 0.4442, Eval Accuracy: 0.7528, Eval Precision: 0.6721, Eval Recall: 0.7555, Eval F1: 0.7113
Epoch: [19/30]: Loss: 0.4722, Accuracy: 0.7930, Precision: 0.6656, Recall: 0.7649, F1: 0.7118

	Loss: 0.4373, Eval Accuracy: 0.7528, Eval Precision: 0.6656, Eval Recall: 0.7649, Eval F1: 0.7118
Epoch: [20/30]: Loss: 0.4673, 

Epoch: [28/30]: Loss: 0.4330, Accuracy: 0.7955, Precision: 0.7175, Recall: 0.7416, F1: 0.7294

	Loss: 0.5445, Eval Accuracy: 0.7528, Eval Precision: 0.7175, Eval Recall: 0.7416, Eval F1: 0.7294
Epoch: [29/30]: Loss: 0.4314, Accuracy: 0.7968, Precision: 0.7143, Recall: 0.7458, F1: 0.7297

	Loss: 0.5431, Eval Accuracy: 0.7528, Eval Precision: 0.7143, Eval Recall: 0.7458, Eval F1: 0.7297
Epoch: [30/30]: Loss: 0.4300, Accuracy: 0.7968, Precision: 0.7110, Recall: 0.7474, F1: 0.7288

	Loss: 0.5415, Eval Accuracy: 0.7528, Eval Precision: 0.7110, Eval Recall: 0.7474, Eval F1: 0.7288
Epoch: [1/30]: Loss: 0.6883, Accuracy: 0.5574, Precision: 0.8339, Recall: 0.4571, F1: 0.5905

	Loss: 0.6890, Eval Accuracy: 0.5955, Eval Precision: 0.8339, Eval Recall: 0.4571, Eval F1: 0.5905
Epoch: [2/30]: Loss: 0.6633, Accuracy: 0.7556, Precision: 0.3746, Recall: 0.9664, F1: 0.5399

	Loss: 0.6693, Eval Accuracy: 0.7079, Eval Precision: 0.3746, Eval Recall: 0.9664, Eval F1: 0.5399
Epoch: [3/30]: Loss: 0.6417, Acc

In [21]:
mean_f_score_skf_torch = get_statistics(list(accuracy.values()), list(precision.values()), list(recall.values()), list(f1.values()))

Stdev:
	Acc: 0.0549
	Pre: 0.0864
	Rec: 0.0795
	F1: 0.0773
Mean Scores:
	Acc: 0.8001
	Pre: 0.6867
	Rec: 0.7690
	F1: 0.7243


In [22]:
print(mean_f_score, mean_f_score_torch)
print(mean_f_score_skf, mean_f_score_skf_torch)

0.7130899101858592 0.7192815072017601
0.7285869206768798 0.7242527604615019
